In [77]:
import pandas as pd
import re, os 
import datetime
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

os.listdir('./data_down')


"""
download data without empty ou, when select
['bca_cc_cust_rel.csv',
 'bca_without_mail.csv',
 'dwd_dsc_d365_contract_df.csv',
 'dwd_dsc_d365_opportunity_df.csv',
 'dwd_dsc_finance_estimate_summary_di.csv',
 'dwd_dsc_hr_cost_dtl.csv',
 'dwd_fact_warehouse_billing_detail_dtl.csv']
"""

"\ndownload data without empty ou, when select\n['bca_cc_cust_rel.csv',\n 'bca_without_mail.csv',\n 'dwd_dsc_d365_contract_df.csv',\n 'dwd_dsc_d365_opportunity_df.csv',\n 'dwd_dsc_finance_estimate_summary_di.csv',\n 'dwd_dsc_hr_cost_dtl.csv',\n 'dwd_fact_warehouse_billing_detail_dtl.csv']\n"

In [78]:
df_oppa = pd.read_csv('./data_down/' + 'dwd_dsc_d365_opportunity_df.csv', sep = '\001') 
df_bill = pd.read_csv('./data_down/' + 'dwd_dsc_finance_estimate_summary_di.csv', sep = '\001') 
df_cost = pd.read_csv('./data_down/' + 'dwd_dsc_hr_cost_dtl.csv', sep = '\001') 
df_esti = pd.read_csv('./data_down/' + 'dwd_fact_warehouse_billing_detail_dtl.csv', sep = '\001') 
rel = pd.read_csv('./data_down/' + 'bca_cc_cust_rel.csv', sep = '\001') 
# for i in os.listdir('./data_down'): 
#     print(re.findall( '\w+', i)[0])
#     df = pd.read_csv('./data_down/' + i, sep = '\001') 
def colname_modi(df):
    df.columns = [ re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
for i in [df_oppa,df_bill,df_cost,df_esti,rel]:
    colname_modi(i)

In [79]:
# pre screen data
df_cost = df_cost.dropna(axis=1, how = 'all').fillna(0)
frame = pd.DataFrame(df_cost.sum(axis = 0) == 0)
df_cost = df_cost.loc[:, frame[frame[0] == False].index.tolist()]
del frame
rel = rel.drop(['cc_in_bca', 'start_date'], axis=1)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_33456/1032199403.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  frame = pd.DataFrame(df_cost.sum(axis = 0) == 0)


In [80]:
def data_prepare():
    """
    data prepare
    """
    df = pd.read_csv('./data_down/dwd_dsc_d365_opportunity_df.csv', sep = '\001')

    df.columns = [re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
    df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
    colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']
    for i in colname:
        df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

    df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
    df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])

    contract_summary = df.groupby([
        'account_name_en', 'contract_end_date', 'account_id','bg', 'contract_term', 'milestone','opportunity_record'\
        ,'product1', 'product2', 'sector'
        ])[[
        'annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue', 'contract_value']].sum()
    contract_summary = contract_summary.reset_index()
    contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)
    return contract_summary

contract_summary = data_prepare()

In [81]:
df_list = [df_oppa,df_bill,df_cost,df_esti]

In [83]:
def check_ouname(df):
    print(pd.Series(df.columns).str.extract('(ou_code|cost_center)').dropna(),'\t')

for i in df_list:
    check_ouname(i)

Empty DataFrame
Columns: [0]
Index: [] 	
              0
13  cost_center 	
         0
1  ou_code 	
              0
12  cost_center 	


In [84]:
def to_cust_level(df):
    """
    merge all to cust level
    """
    try:
        df = df.merge(rel, left_on = 'ou_code', right_on = 'cc_in_ou', how  ='left')
    except:
        df = df.merge(rel, left_on = 'cost_center', right_on = 'cc_in_ou', how  ='left')
    

    return df

In [87]:
# to_cust_level(df_bill)[~to_cust_level(df_bill)['account_name_en'].isna()]

In [88]:
contract_summary.head()

,account_name_en,contract_end_date,account_id,bg,contract_term,milestone,opportunity_record,product1,product2,sector,annual_average_gross_profit_base,annual_average_gross_profit,annual_average_revenue_base,annual_average_revenue,contract_value,contract_end_yr
0,ABC,2021-06-28,e0209d33-e3aa-ea11-a812-000d3a851174,TSC,12.0,Closed-Canceled,New Business,Warehousing,Transportation,Technology,5000.0,5000.0,50000.0,50000.0,50000.00,2021
1,ADIDAS,2018-09-29,f64a8fa5-a304-4c85-9641-7205e77e70d8,HK,36.0,Contract Signed,New Business,Warehousing,Warehousing,Retail,950.4,1188.0,10777.6,13472.0,40416.00,2018
2,ADIDAS,2019-10-30,f64a8fa5-a304-4c85-9641-7205e77e70d8,HK,12.0,Contract Signed,Straightforward Renewals,Warehousing,Transportation,Retail,1087.2,1359.0,13123.2,16404.0,0.00,2019
3,ADIDAS,2020-12-02,f64a8fa5-a304-4c85-9641-7205e77e70d8,HK,6.0,Contract Signed,New Business,Warehousing,Transportation,Retail,1.6,2.0,1728.0,2160.0,1080.00,2020
4,ADIDAS,2021-06-01,f64a8fa5-a304-4c85-9641-7205e77e70d8,HK,6.0,Closed-Lost,Change Renewals,Transportation,Warehousing,Retail,1.6,2.0,1728.0,2160.0,1182.82,2021


In [ ]:
# def get_options(list_stocks):
#     dict_list = []
#     for i in list_stocks:
#         dict_list.append({'label': i, 'value': i})
#     return dict_list

In [ ]:
df_sub = contract_summary

In [ ]:
df_sub[df_sub['account_name_en'] == 'ABC']['annual_average_revenue']

In [ ]:
min = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
max = pd.to_datetime('2024-01-01', format='%Y-%m-%d')

In [ ]:
# fig = go.Figure(data = [go.Scatter(x = df_sub ['contract_end_date'] ,
#                                 y = df_sub ['annual_average_revenue'],
#                                 mode='markers',
#                                 marker_size=df_sub['annual_average_revenue']/1000,
#                                 opacity=0.87,
#                                 name='account',
#                                 textposition='bottom center')],
#                                 layout=go.Layout(
#                                     xaxis = dict(range = [min, max]),
#                                     yaxis = dict(range = [0, 100000]),
#                                     legend = {'xanchor': 'left'}
#                                 )
#                                 ) 
# fig

In [ ]:
# df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
# colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']
# for i in colname:
#     df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

# df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
# df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])
# contract_summary = df.groupby(['account_name_en', 'contract_end_date'])[[
#        'annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']].sum()
# contract_summary = contract_summary.reset_index()
# contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)

In [ ]:
contract_summary['contract_end_date']

In [ ]:
# fig = px.scatter(x = "contract_end_date", y= "annual_average_revenue", \
#     color = 'account_name_en',  size = 'annual_average_revenue',
#     data_frame  = contract_summary.head(65))

# min = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
# max = pd.to_datetime('2024-01-01', format='%Y-%m-%d')
# fig.update_xaxes(range = list([min, max]))

# fig.update_layout(
#     title_text="Title",
#     showlegend = True
# )

# # Add range slider
# fig.update_layout(

# )

